In [1]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import folium

%matplotlib inline
pd.set_option('display.max_columns', 500)

In [2]:
applications = pd.read_csv('../data/pruned_data.csv', low_memory=False)
applications

,Unnamed: 0,lei,derived_msa_md,state_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,total_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_observed,applicant_race_observed,applicant_age,aus_1,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,0,RVDPPPGHCGZ40J4VQ731,26420,TX,4.820155e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Asian,Male,1,0,2,1,1,1,2,2,215000,97.0,6.5,-0.034,7436.14,NaN,4336.0,NaN,2703.37,360,NaN,2,2,2,2,225000,1,59.0,42,2,10,2,2,25-34,1,10,NaN,NaN,NaN,6907,71.83,90800,80.0,1409,2002,28
1,1,RVDPPPGHCGZ40J4VQ731,19124,TX,4.808503e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Male,1,2,2,2,1,1,2,2,445000,92.74,6.75,0.847,16155.19,NaN,4290.4,3276.4,NaN,360,NaN,2,2,2,2,475000,1,202.0,46,9,2,2,2,45-54,3,10,NaN,NaN,NaN,6001,39.96,97400,150.0,1255,1431,14
2,2,RVDPPPGHCGZ40J4VQ731,40900,CA,6.017030e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Female,4,0,2,1,1,1,2,2,235000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360,NaN,2,2,2,2,NaN,1,138.0,NaN,9,9,2,2,35-44,6,10,NaN,NaN,NaN,2703,39.92,102400,60.0,478,1631,51
3,3,RVDPPPGHCGZ40J4VQ731,25540,CT,9.013538e+09,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,1,2,2,2,1,1,2,2,425000,91.47,7.125,1.48,22789.69,NaN,11771.13,NaN,5624.46,360,NaN,2,2,2,2,455000,1,150.0,46,3,10,2,2,45-54,3,10,NaN,NaN,NaN,3948,8.61,113200,121.0,1271,1507,42
4,4,RVDPPPGHCGZ40J4VQ731,12580,MD,2.451015e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Female,4,0,2,2,1,1,2,2,205000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360,NaN,2,2,2,2,NaN,1,75.0,NaN,9,9,2,2,35-44,3,10,NaN,NaN,NaN,3249,92.34,116100,38.0,444,1500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12974034,16080204,RVDPPPGHCGZ40J4VQ731,38060,AZ,4.013422e+09,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,White,Male,1,2,2,2,1,1,2,2,295000,96.5,6.125,0.414,15055.59,NaN,5856.47,5856.47,NaN,360,NaN,2,2,2,2,305000,1,61.0,42,3,10,2,2,25-34,3,10,NaN,NaN,NaN,3994,85.80,88800,53.0,430,1296,50
12974035,16080205,RVDPPPGHCGZ40J4VQ731,12060,GA,1.311713e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,1,1,2,1,1,1,2,2,645000,88.12,6.25,-0.492,9355.98,NaN,4990.25,NaN,3857.1,360,60,2,2,2,2,735000,1,201.0,39,1,10,2,2,35-44,1,10,NaN,NaN,NaN,4745,34.77,95700,166.0,1380,1497,17
12974036,16080206,RVDPPPGHCGZ40J4VQ731,29820,NV,3.200300e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,White,Male,3,0,2,2,1,1,2,2,275000,96.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360,NaN,2,2,2,2,285000,1,58.0,50%-60%,3,10,2,2,35-44,3,9,NaN,NaN,NaN,5249,84.82,81700,50.0,388,1007,36
12974037,16080207,RVDPPPGHCGZ40J4VQ731,39340,UT,4.904901e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Joint,1,2,2,2,32,1,2,2,515000,78.81,6.125,0.049,2166

In [3]:
tracts = gpd.read_file('../data/tract.json')
ct_tracts = pd.read_csv('../data/2022tractcrosswalk.csv', usecols=[0,1])
ct_tracts['tract_fips_2020'] = ['0' + str(tract) for tract in ct_tracts['tract_fips_2020']]
ct_tracts['Tract_fips_2022'] = ['0' + str(tract) for tract in ct_tracts['Tract_fips_2022']]
tract_mapping = {row['Tract_fips_2022']:row['tract_fips_2020'] for ind, row in ct_tracts.iterrows()}
tracts['GEOID'] = tracts['GEOID'].replace(tract_mapping)
tracts

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,01,097,006505,1400000US01097006505,01097006505,65.05,Census Tract 65.05,AL,Mobile County,Alabama,CT,94686529,501122,"POLYGON ((-88.40601 30.58976, -88.38573 30.615..."
1,05,131,001202,1400000US05131001202,05131001202,12.02,Census Tract 12.02,AR,Sebastian County,Arkansas,CT,4381207,34047,"POLYGON ((-94.43277 35.33941, -94.39136 35.338..."
2,06,059,099511,1400000US06059099511,06059099511,995.11,Census Tract 995.11,CA,Orange County,California,CT,944442,1356526,"POLYGON ((-118.11464 33.74461, -118.11305 33.7..."
3,06,037,620904,1400000US06037620904,06037620904,6209.04,Census Tract 6209.04,CA,Los Angeles County,California,CT,576178,413509,"POLYGON ((-118.41269 33.88388, -118.40768 33.8..."
4,06,001,428500,1400000US06001428500,06001428500,4285,Census Tract 4285,CA,Alameda County,California,CT,624197,383522,"POLYGON ((-122.26360 37.76743, -122.25575 37.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85188,37,159,051502,1400000US37159051502,37159051502,515.02,Census Tract 515.02,NC,Rowan County,North Carolina,CT,13653797,602104,"POLYGON ((-80.69609 35.50581, -80.65807 35.546..."
85189,06,065,042802,1400000US06065042802,06065042802,428.02,Census Tract 428.02,CA,Riverside County,California,CT,3156976,0,"POLYGON ((-117.24372 33.78334, -117.23477 33.7..."
85190,48,423,001102,1400000US48423001102,48423001102,11.02,Census Tract 11.02,TX,Smith County,Texas,CT,2774447,679,"POLYGON ((-95.30362 32.31181, -95.27895 32.312..."
85191,36,015,000500,1400000US36015000500,36015000500,5,Census Tract 5,NY,Chemung County,New York,CT,1493597,50755,"POLYGON ((-76.83284 42.08993, -76.82248 42.096..."


In [4]:
applications['census_string'] = [str(tract)[:-2] for tract in applications['census_tract']]
applications['census_string'] = ['0'+ tract if len(tract) == 10 else tract for tract in applications['census_string']]
applications.head()

,Unnamed: 0,lei,derived_msa_md,state_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,total_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_observed,applicant_race_observed,applicant_age,aus_1,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,census_string
0,0,RVDPPPGHCGZ40J4VQ731,26420,TX,4.820155e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Asian,Male,1,0,2,1,1,1,2,2,215000,97.0,6.5,-0.034,7436.14,NaN,4336.0,NaN,2703.37,360,NaN,2,2,2,2,225000,1,59.0,42,2,10,2,2,25-34,1,10,NaN,NaN,NaN,6907,71.83,90800,80.0,1409,2002,28,48201552502
1,1,RVDPPPGHCGZ40J4VQ731,19124,TX,4.808503e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Male,1,2,2,2,1,1,2,2,445000,92.74,6.75,0.847,16155.19,NaN,4290.4,3276.4,NaN,360,NaN,2,2,2,2,475000,1,202.0,46,9,2,2,2,45-54,3,10,NaN,NaN,NaN,6001,39.96,97400,150.0,1255,1431,14,48085031314
2,2,RVDPPPGHCGZ40J4VQ731,40900,CA,6.017030e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Female,4,0,2,1,1,1,2,2,235000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360,NaN,2,2,2,2,NaN,1,138.0,NaN,9,9,2,2,35-44,6,10,NaN,NaN,NaN,2703,39.92,102400,60.0,478,1631,51,06017030301
3,3,RVDPPPGHCGZ40J4VQ731,25540,CT,9.013538e+09,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,1,2,2,2,1,1,2,2,425000,91.47,7.125,1.48,22789.69,NaN,11771.13,NaN,5624.46,360,NaN,2,2,2,2,455000,1,150.0,46,3,10,2,2,45-54,3,10,NaN,NaN,NaN,3948,8.61,113200,121.0,1271,1507,42,09013538202
4,4,RVDPPPGHCGZ40J4VQ731,12580,MD,2.451015e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Female,4,0,2,2,1,1,2,2,205000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360,NaN,2,2,2,2,NaN,1,75.0,NaN,9,9,2,2,35-44,3,10,NaN,NaN,NaN,3249,92.34,116100,38.0,444,1500,0,24510150400


In [5]:
approved = applications[(applications['action_taken'] == 1)]
med_loan_value = approved.groupby('census_string')['loan_amount'].median()
med_loan_value = med_loan_value.reset_index()
med_loan_value.sort_values(by='loan_amount',ascending=False)

,census_string,loan_amount
19380,12099003507,5005000.0
16590,12021000500,5005000.0
11214,06075060100,4405000.0
7652,06037700600,3855000.0
51157,36061009903,3825000.0
...,...,...
37477,26049001700,15000.0
27139,17163500900,15000.0
27057,17143005100,15000.0
68363,47157011401,5000.0


In [14]:
m = folium.Map(location=[40, -95], zoom_start=4, prefer_canvas=True)

folium.Choropleth(
    geo_data=tracts,
    name="choropleth",
    data=med_loan_value,
    columns=["census_string", "loan_amount"],
    key_on="feature.properties.GEOID",
    fill_color="PuBu",
    fill_opacity=0.7,
    nan_fill_opacity = 0,
    line_opacity=.01,
    legend_name="Loan Amount",
    bins=[0,250_000,500_000,1_000_000,2_000_000,5_050_000]
).add_to(m)

folium.LayerControl().add_to(m)

m.save('../maps/loan_value.html')

In [7]:
applications = applications[applications['rate_spread'] != 'Exempt']

In [8]:
applications['rate_spread'] = applications['rate_spread'].astype('float')

/tmp/ipykernel_5485/1542282059.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  applications['rate_spread'] = applications['rate_spread'].astype('float')


In [9]:
spreads = applications.groupby('census_string')['rate_spread'].median()

In [15]:
m = folium.Map(location=[40, -95], zoom_start=4, prefer_canvas=True)

folium.Choropleth(
    geo_data=tracts,
    name="choropleth",
    data=spreads,
    columns=["census_string", "rate_spread"],
    key_on="feature.properties.GEOID",
    fill_color="PuBu",
    fill_opacity=0.7,
    nan_fill_opacity = 0,
    line_opacity=.01,
    legend_name="Loan Amount",
    bins=[-5,-1,-0.5,0,0.5,1,2,3,5,10]
).add_to(m)

folium.LayerControl().add_to(m)

m.save('../maps/rate_spread.html')

In [11]:
applications[applications['census_string'] == '36081056100']

,Unnamed: 0,lei,derived_msa_md,state_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,total_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_observed,applicant_race_observed,applicant_age,aus_1,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,census_string
12664538,15601502,2549006T9J2DMLJVRS64,35614,NY,3.608106e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Race Not Available,Sex Not Available,2,0,2,1,2,1,2,1,255000,42.338,2.259,4.8,NaN,NaN,NaN,NaN,NaN,360,12,2,1,2,2,595000,1,88.0,45,8,10,2,2,55-64,6,10,NaN,NaN,NaN,20,50.0,99000,0.0,4,0,0,36081056100


In [12]:
applications['census_string'].value_counts()

census_string
n              132759
48157673202      3170
48201542902      3149
48029172002      3138
45015020711      2727
                ...  
39065000401         1
39049002710         1
21067000801         1
15003008502         1
06037209401         1
Name: count, Length: 83917, dtype: int64